# 커스텀 도구

In [2]:
import os
from langchain.tools import tool
import requests

@tool(
        "weather_lookup",
        description="지정된 도시의 실시간 날씨 정보를 조회합니다.",
        return_direct=True  # 결과를 에이전트를 거치지 않고 바로 반환
)
def get_weather(location: str) -> str:
    """주어진 위치의 현재 날씨 정보를 조회합니다.
    
    Args:
        location: 날씨를 조회할 도시 이름 (영어)

    Returns:
        날씨 정보 문자열 (설명, 온도 포함)
    """

    api_key = os.getenv('OPENWEATHERMAP_API_KEY')

    if not api_key:
        raise ValueError("OpenWeatherMap API 키가 설정되지 않았습니다.")

    base_url = "http://api.openweathermap.org/data/2.5/weather"

    # 입력 전처리
    location = location.strip("'\'")

    # API 요청 파라미터 설정
    params = {
        "q": location,
        "appid": api_key,
        "units": "metric",
        "lang": "kr"
    }

    # API 요청 및 응답 처리
    response = requests.get(base_url, params=params)
    response.raise_for_status()

    data = response.json()
    weather_description = data["weather"][0]["description"]
    temperature = data["main"]["temp"]
    humidity = data["main"]["humidity"]

    return (
        f"{location}의 현재 날씨:\n"
        f"- 상태: {weather_description}\n"
        f"- 온도: {temperature}°C\n"
        f"- 습도: {humidity}%"
    )